
Validation Script: dscim-new vs Original dscim

This script validates the dscim-new refactored implementation against the original
dscim library using the dummy data from dscim-testing/.

It is designed to be run interactively, similar to a notebook, with each section
executed sequentially.

Expected Outputs:
-----------------
The script will:
1. Load dummy sectoral damages data
2. Generate damage function coefficients
3. Calculate marginal damages using FAIR projections
4. Compute global consumption with extrapolation
5. Calculate discount factors
6. Compute final SCC values

The outputs will be compared against the reference outputs in:
    dscim-testing/dummy_data/results/AR6_ssp/dummy_not_coastl_sector/2020/unmasked/

Usage:
------
Run this file interactively or execute it as a script:
    python examples/validate_against_original.py




In [1]:
%load_ext autoreload

In [2]:

import sys
from pathlib import Path
import numpy as np
import xarray as xr
import pandas as pd
import yaml
from datetime import datetime

project_root = Path().resolve()

from dscim_new.preprocessing.damage_functions import DamageFunctionProcessor
from dscim_new.core.damage_functions import calculate_marginal_damages_from_fair
from dscim_new.preprocessing.consumption import extrapolate_global_consumption
from dscim_new.core.discounting import calculate_stream_discount_factors_per_scenario
from dscim_new.core.scc import calculate_scc_with_uncertainty, summarize_scc
from dscim_new.config.schemas import (
    DamageFunctionConfig,
    ClimateDataConfig,
    DiscountingConfig
)

In [3]:
# =============================================================================
# CONFIGURATION
# =============================================================================
print("="*80)
print("DSCIM-NEW VALIDATION AGAINST ORIGINAL DSCIM")
print("="*80)
print(f"Start time: {datetime.now()}")
print("="*80)

# Configuration matching run_integration_result.py
sector_name = "dummy_not_coastl_sector"
recipe = "risk_aversion"
eta = 2.0
rho = 0.0001
pulse_year = 2020
discount_type = "naive_ramsey"

print(f"\nConfiguration:")
print(f"  Sector: {sector_name}")
print(f"  Recipe: {recipe}")
print(f"  Discount Type: {discount_type}")
print(f"  Eta: {eta}, Rho: {rho}")
print(f"  Pulse Year: {pulse_year}")

DSCIM-NEW VALIDATION AGAINST ORIGINAL DSCIM
Start time: 2025-10-08 02:06:10.430832

Configuration:
  Sector: dummy_not_coastl_sector
  Recipe: risk_aversion
  Discount Type: naive_ramsey
  Eta: 2.0, Rho: 0.0001
  Pulse Year: 2020


In [4]:
# =============================================================================
# STEP 1: LOAD CONFIGURATION
# =============================================================================
print(f"\n{datetime.now()}: STEP 1 - Loading Configuration")
print("-"*80)

config_path = project_root.parent.parent  / "dscim-testing" / "configs" / "dummy_config.yaml"
print(f"Loading configuration from: {config_path}")

with open(config_path, "r") as f:
    conf = yaml.safe_load(f)

print("✓ Configuration loaded")



2025-10-08 02:06:10.433433: STEP 1 - Loading Configuration
--------------------------------------------------------------------------------
Loading configuration from: /Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/configs/dummy_config.yaml
✓ Configuration loaded


In [6]:
# =============================================================================
# STEP 2: LOAD SECTORAL DAMAGES
# =============================================================================
print(f"\n{datetime.now()}: STEP 2 - Loading Sectoral Damages")
print("-"*80)

sector_config = conf["sectors"][sector_name]
sector_path = project_root.parent.parent / "dscim-testing" / sector_config["sector_path"]

print(f"Loading sectoral damages from: {sector_path}")
sectoral_damages = xr.open_zarr(sector_path)

print(f"  Loaded damages with dimensions: {sectoral_damages.dims}")
print(f"  Variables: {list(sectoral_damages.data_vars)}")

# Extract damage variable
if "damages" in sectoral_damages:
    damages = sectoral_damages["damages"]
else:
    data_vars = list(sectoral_damages.data_vars)
    if data_vars:
        damages = sectoral_damages[data_vars[0]]
        print(f"  Using damage variable: {data_vars[0]}")
    else:
        raise ValueError("No damage variable found in sectoral data")

print(f"  Damage dimensions: {damages.dims}")
print(f"  Damage shape: {damages.shape}")
print(f"  Damage coords: {list(damages.coords.keys())}")



2025-10-08 02:06:51.748785: STEP 2 - Loading Sectoral Damages
--------------------------------------------------------------------------------
Loading sectoral damages from: /Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/dummy_data/sectoral/not_coastl_damages.zarr
  Loaded damages with dimensions: FrozenMappingWarningOnValuesAccess({'rcp': 2, 'region': 2, 'gcm': 2, 'year': 11, 'model': 2, 'ssp': 5, 'batch': 15})
  Variables: ['delta_dummy', 'histclim_dummy']
  Using damage variable: delta_dummy
  Damage dimensions: ('rcp', 'region', 'gcm', 'year', 'model', 'ssp', 'batch')
  Damage shape: (2, 2, 2, 11, 2, 5, 15)
  Damage coords: ['batch', 'gcm', 'model', 'rcp', 'region', 'ssp', 'year']


In [7]:
# =============================================================================
# STEP 3: LOAD CLIMATE DATA
# =============================================================================
print(f"\n{datetime.now()}: STEP 3 - Loading Climate Data")
print("-"*80)

base_path = project_root.parent.parent / "dscim-testing"

# Load GMST data (CSV)
gmst_path = base_path / conf["AR6_ssp_climate"]["gmst_path"]
print(f"Loading GMST data from: {gmst_path}")
gmst_df = pd.read_csv(gmst_path)
print(f"  GMST shape: {gmst_df.shape}")
print(f"  GMST columns: {list(gmst_df.columns)}")

# Load GMSL data (Zarr)
gmsl_path = base_path / conf["AR6_ssp_climate"]["gmsl_path"]
print(f"Loading GMSL data from: {gmsl_path}")
gmsl_data = xr.open_zarr(gmsl_path)
print(f"  GMSL dims: {gmsl_data.dims}")
print(f"  GMSL coords: {list(gmsl_data.coords.keys())}")

# Load FAIR temperature projections
fair_temp_path = base_path / conf["AR6_ssp_climate"]["gmst_fair_path"]
print(f"Loading FAIR temperature from: {fair_temp_path}")
fair_temp = xr.open_dataset(fair_temp_path)
print(f"  FAIR temp dims: {fair_temp.dims}")
print(f"  FAIR temp coords: {list(fair_temp.coords.keys())}")

# Load FAIR GMSL projections
fair_gmsl_path = base_path / conf["AR6_ssp_climate"]["gmsl_fair_path"]
print(f"Loading FAIR GMSL from: {fair_gmsl_path}")
fair_gmsl = xr.open_dataset(fair_gmsl_path)
print(f"  FAIR GMSL dims: {fair_gmsl.dims}")

# Load pulse conversion factors
conversion_path = base_path / conf["AR6_ssp_climate"]["damages_pulse_conversion_path"]
print(f"Loading conversion factors from: {conversion_path}")
conversion = xr.open_dataset(conversion_path)
print(f"  Conversion dims: {conversion.dims}")

# Combine FAIR temperature and GMSL into single datasets
fair_combined = xr.merge([fair_temp, fair_gmsl])

print("✓ Climate data loaded")
print(f"  FAIR combined variables: {list(fair_combined.data_vars)}")
print(f"  FAIR combined dims: {fair_combined.dims}")

# Separate control and pulse scenarios (following original dscim approach)
# See: dscim/src/dscim/menu/simple_storage.py:193-202
print("\nSeparating control and pulse scenarios...")

# Determine which variables are in the dataset
anomaly_vars = []
if any('temperature' in v for v in fair_combined.data_vars):
    anomaly_vars.append('temperature')
if any('gmsl' in v for v in fair_combined.data_vars):
    anomaly_vars.append('gmsl')

print(f"  Anomaly variables detected: {anomaly_vars}")

# Create fair_control: select control_* variables and rename
control_var_names = [f"control_{var}" for var in anomaly_vars if f"control_{var}" in fair_combined.data_vars]
if not control_var_names:
    # Try medianparams_control_*
    control_var_names = [f"medianparams_control_{var}" for var in anomaly_vars if f"medianparams_control_{var}" in fair_combined.data_vars]

print(f"  Control variable names: {control_var_names}")

fair_control = fair_combined[control_var_names]
# Rename: remove control_ or medianparams_control_ prefix
fair_control = fair_control.rename({v: v.replace('control_', '').replace('medianparams_', '') for v in control_var_names})

# Create fair_pulse: select pulse_* variables and rename
pulse_var_names = [f"pulse_{var}" for var in anomaly_vars if f"pulse_{var}" in fair_combined.data_vars]
if not pulse_var_names:
    # Try medianparams_pulse_*
    pulse_var_names = [f"medianparams_pulse_{var}" for var in anomaly_vars if f"medianparams_pulse_{var}" in fair_combined.data_vars]

print(f"  Pulse variable names: {pulse_var_names}")

fair_pulse = fair_combined[pulse_var_names]
# Rename: remove pulse_ or medianparams_pulse_ prefix
fair_pulse = fair_pulse.rename({v: v.replace('pulse_', '').replace('medianparams_', '') for v in pulse_var_names})

print(f"\n  Fair control variables after rename: {list(fair_control.data_vars)}")
print(f"  Fair pulse variables after rename: {list(fair_pulse.data_vars)}")



2025-10-08 02:06:59.631627: STEP 3 - Loading Climate Data
--------------------------------------------------------------------------------
Loading GMST data from: /Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/dummy_data/climate/GMTanom_all_temp_2001_2010_smooth.csv
  GMST shape: (44, 4)
  GMST columns: ['year', 'rcp', 'gcm', 'anomaly']
Loading GMSL data from: /Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/dummy_data/climate/coastal_gmsl_v0.20.zarr
  GMSL dims: FrozenMappingWarningOnValuesAccess({'year': 11, 'slr': 2})
  GMSL coords: ['slr', 'year']
Loading FAIR temperature from: /Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/dummy_data/climate/ar6_fair162_sim.nc
  FAIR temp dims: FrozenMappingWarningOnValuesAccess({'year': 99, 'rcp': 2, 'simulation': 4, 'gas': 1, 'pulse_year': 1})
  FAIR temp coords: ['year', 'rcp', 'simulation', 'gas', 'pulse_year']
Loading FAIR GMSL from: /Users/sebastiancadavids

In [8]:
# =============================================================================
# STEP 4: LOAD ECONOMIC DATA
# =============================================================================
print(f"\n{datetime.now()}: STEP 4 - Loading Economic Data")
print("-"*80)

econ_path = project_root.parent.parent / "dscim-testing" / conf["econdata"]["global_ssp"]
print(f"Loading economic data from: {econ_path}")
econ_data = xr.open_zarr(econ_path)

print(f"  Economic data dims: {econ_data.dims}")
print(f"  Variables: {list(econ_data.data_vars)}")

# Extract GDP and population
gdp = econ_data["gdp"] if "gdp" in econ_data else econ_data["GDP"]
pop = econ_data["pop"] if "pop" in econ_data else econ_data["population"]

print(f"  GDP shape: {gdp.shape}")
print(f"  Population shape: {pop.shape}")
print("✓ Economic data loaded")


2025-10-08 02:07:00.683262: STEP 4 - Loading Economic Data
--------------------------------------------------------------------------------
Loading economic data from: /Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/dummy_data/econ/integration-econ-bc39.zarr
  Economic data dims: FrozenMappingWarningOnValuesAccess({'year': 11, 'ssp': 5, 'region': 2, 'model': 2})
  Variables: ['gdp', 'gdppc', 'pop']
  GDP shape: (11, 5, 2, 2)
  Population shape: (11, 5, 2, 2)
✓ Economic data loaded


In [9]:
# =============================================================================
# STEP 5: INSPECT DIMENSION NAMES
# =============================================================================
print(f"\n{datetime.now()}: STEP 5 - Inspecting Dimension Names")
print("-"*80)

print("\nDamage dimensions:")
print(f"  dims: {damages.dims}")
print(f"  coords: {list(damages.coords.keys())}")

print("\nGMST dimensions (from DataFrame):")
print(f"  columns: {list(gmst_df.columns)}")

print("\nGMSL dimensions:")
print(f"  dims: {gmsl_data.dims}")
print(f"  coords: {list(gmsl_data.coords.keys())}")

# Determine the actual dimension names used in the data
# The error shows: "dimensions FrozenMappingWarningOnValuesAccess({'year': 11, 'rcp': 2, 'gcm': 2})"
# So the climate data uses 'rcp' and 'gcm' instead of 'ssp' and 'model'

# Map dimension names
damage_ssp_dim = None
damage_model_dim = None
damage_year_dim = None

for dim in damages.dims:
    if 'ssp' in dim.lower() or 'rcp' in dim.lower() or 'scenario' in dim.lower():
        damage_ssp_dim = dim
    if 'model' in dim.lower() or 'gcm' in dim.lower():
        damage_model_dim = dim
    if 'year' in dim.lower():
        damage_year_dim = dim

print(f"\nDetected dimension names in damages:")
print(f"  SSP/Scenario dimension: {damage_ssp_dim}")
print(f"  Model dimension: {damage_model_dim}")
print(f"  Year dimension: {damage_year_dim}")

print(f"\n⚠ NOTE: The climate data dimensions determine what the rolling window will use.")
print(f"  Climate (GMST) has: {list(gmst_df.columns)}")
print(f"  When converted to xarray, it will likely have: year, rcp, gcm")



2025-10-08 02:07:01.719854: STEP 5 - Inspecting Dimension Names
--------------------------------------------------------------------------------

Damage dimensions:
  dims: ('rcp', 'region', 'gcm', 'year', 'model', 'ssp', 'batch')
  coords: ['batch', 'gcm', 'model', 'rcp', 'region', 'ssp', 'year']

GMST dimensions (from DataFrame):
  columns: ['year', 'rcp', 'gcm', 'anomaly']

GMSL dimensions:
  dims: FrozenMappingWarningOnValuesAccess({'year': 11, 'slr': 2})
  coords: ['slr', 'year']

Detected dimension names in damages:
  SSP/Scenario dimension: ssp
  Model dimension: model
  Year dimension: year

⚠ NOTE: The climate data dimensions determine what the rolling window will use.
  Climate (GMST) has: ['year', 'rcp', 'gcm', 'anomaly']
  When converted to xarray, it will likely have: year, rcp, gcm


In [10]:
# =============================================================================
# STEP 6: GENERATE DAMAGE FUNCTION COEFFICIENTS
# =============================================================================
print(f"\n{datetime.now()}: STEP 6 - Generating Damage Function Coefficients")
print("-"*80)

# Get formula from config
formula = sector_config["formula"]
print(f"Formula: {formula}")

# Create damage function config
# NOTE: We need to pass the correct dimension names to the rolling window function
df_config = DamageFunctionConfig(
    formula=formula,
    fit_type="ols",
    fit_method="rolling_window",  # Original dscim uses rolling window
    window_size=5,
    year_range=(2020, 2101),
    extrapolation_method="global_c_ratio",
    save_points=True
)

# Create climate config
climate_config = ClimateDataConfig(
    gmst_path=None,  # Will pass data directly
    gmsl_path=None
)

# Create processor
processor = DamageFunctionProcessor(
    config=df_config,
    climate_config=climate_config,
    verbose=True
)

print(f"Generating damage function with discount_type: {discount_type}")

# The processor will auto-detect dimension names (rcp/gcm vs ssp/model)
# and pass them to the rolling window function

# Generate damage function
result = processor.generate_damage_function(
    damages=damages,
    sector=sector_name,
    pulse_year=pulse_year,
    climate_data=(gmst_df, gmsl_data),
    discount_type=discount_type,
    save_outputs=False
)

damage_function_coefficients = result.coefficients_grid

print(f"\n✓ Damage function coefficients generated")
print(f"  Dimensions: {damage_function_coefficients.dims}")
print(f"  Shape: {damage_function_coefficients.sizes}")
print(f"  Variables: {list(damage_function_coefficients.data_vars)}")



2025-10-08 02:07:03.095185: STEP 6 - Generating Damage Function Coefficients
--------------------------------------------------------------------------------
Formula: damages ~ -1 + anomaly + np.power(anomaly, 2)
Generating damage function with discount_type: naive_ramsey


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Generating damage function for dummy_not_coastl_sector, pulse year 2020                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Fitting damage functions with rolling window approach (per year/scenario)

Detected dimensions from climate data: ssp=rcp, model=gcm, year=year

Detected region dimension from damages: region=region

...

Final dimensions: FrozenMappingWarningOnValuesAccess({'discount_type': 1, 'rcp': 2, 'gcm': 2, 'year': 81})

Calculating marginal damages

Generating 100 evaluation points for visualization

✓ Damage function generation complete


✓ Damage function coefficients generated
  Dimensions: FrozenMappingWarningOnValuesAccess({'discount_type': 1, 'rcp': 2, 'gcm': 2, 'year': 81})
  Shape: Frozen({'discount_type': 1, 'rcp': 2, 'gcm': 2, 'year': 81})
  Variables: ['anomaly', 'np.power(anomaly, 2)']


In [11]:
# =============================================================================
# STEP 7: CALCULATE GLOBAL CONSUMPTION (MUST COME BEFORE MARGINAL DAMAGES)
# =============================================================================
print(f"\n{datetime.now()}: STEP 7 - Calculating Global Consumption")
print("-"*80)

# Get end year from config
ext_end_year = conf["global_parameters"]["ext_end_year"]
print(f"Extrapolation end year: {ext_end_year}")

# Determine actual year range available in data
available_years = sorted(gdp.year.values)
print(f"Available years in GDP data: {available_years[0]} to {available_years[-1]}")

# Use the last available years for extrapolation
# Original dscim uses years 2085-2099, but we'll use what's available
if len(available_years) >= 2:
    # Use last ~15 years of data for extrapolation
    end_year = int(available_years[-1])
    start_year = max(int(available_years[0]), end_year - 14)
    print(f"Using extrapolation window: {start_year} to {end_year}")
else:
    raise ValueError(f"Not enough years in GDP data: {available_years}")

global_consumption = extrapolate_global_consumption(
    gdp=gdp,
    pop=pop,
    discount_type=discount_type,
    start_year=start_year,
    end_year=end_year,
    target_year=ext_end_year,
    method="growth_constant"
)

print(f"\n✓ Global consumption calculated")
print(f"  Dimensions: {global_consumption.dims}")
print(f"  Shape: {global_consumption.sizes}")



2025-10-08 02:07:14.241203: STEP 7 - Calculating Global Consumption
--------------------------------------------------------------------------------
Extrapolation end year: 2100
Available years in GDP data: 2020 to 2030
Using extrapolation window: 2020 to 2030

✓ Global consumption calculated
  Dimensions: ('discount_type', 'year', 'ssp', 'model')
  Shape: Frozen({'discount_type': 1, 'year': 81, 'ssp': 5, 'model': 2})


In [20]:
# =============================================================================
# STEP 8: CALCULATE MARGINAL DAMAGES WITH FAIR AGGREGATION
# =============================================================================
print(f"\n{datetime.now()}: STEP 8 - Calculating Marginal Damages")
print("-"*80)

# Get parameters from config
weitzman_params = conf["global_parameters"]["weitzman_parameter"]
fair_agg = conf["global_parameters"]["fair_aggregation"]

# Filter aggregation methods:
# - 'median' is computed POST-SCC, not in marginal damages
# - 'median_params' requires separate FAIR scenario (not yet implemented)
fair_agg_for_marginal_damages = [agg for agg in fair_agg if agg not in ['median', 'median_params']]
print(f"Weitzman parameters: {weitzman_params}")
print(f"FAIR aggregation for marginal damages: {fair_agg_for_marginal_damages}")
print(f"Full FAIR aggregation (including median): {fair_agg}")

# Calculate marginal damages with proper FAIR aggregation
# The function will auto-detect which dimensions to collapse (gcm, simulation, etc.)
marginal_damages = calculate_marginal_damages_from_fair(
    fair_control=fair_control,
    fair_pulse=fair_pulse,
    damage_function_coefficients=damage_function_coefficients,
    formula=formula,
    global_consumption=global_consumption,
    pulse_conversion_factor=1.0,
    fair_aggregation=fair_agg_for_marginal_damages,
    fair_dims=None,  # Auto-detect
    weitzman_parameters=weitzman_params,
    eta=eta
)

print(f"\n✓ Marginal damages calculated")
print(f"  Dimensions: {marginal_damages.dims}")
print(f"  Shape: {marginal_damages.sizes}")


2025-10-08 02:12:20.179652: STEP 8 - Calculating Marginal Damages
--------------------------------------------------------------------------------
Weitzman parameters: [0.1, 1.0, 0.25, 0.01, 0.001]
FAIR aggregation for marginal damages: ['ce', 'mean', 'gwr_mean']
Full FAIR aggregation (including median): ['ce', 'median', 'mean', 'gwr_mean', 'median_params']
Auto-detected FAIR uncertainty dimensions: ['simulation']

✓ Marginal damages calculated
  Dimensions: ('fair_aggregation', 'weitzman_parameter', 'discount_type', 'year', 'ssp', 'model', 'rcp', 'gcm', 'gas', 'pulse_year', 'pulse_years')
  Shape: Frozen({'fair_aggregation': 3, 'weitzman_parameter': 5, 'discount_type': 1, 'year': 100, 'ssp': 5, 'model': 2, 'rcp': 2, 'gcm': 2, 'gas': 1, 'pulse_year': 1, 'pulse_years': 1})


/Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim_new/core/damage_functions.py:1158: FutureWarning: In a future version of xarray the default value for join will change from join='outer' to join='exact'. This change will result in the following ValueError: cannot be aligned with join='exact' because index/labels/sizes are not equal along these coordinates (dimensions): 'year' ('year',) The recommendation is to set join explicitly for this case.
  marginal_damages = xr.concat(marginal_damages_list, dim="fair_aggregation")


In [21]:
# =============================================================================
# STEP 9: CALCULATE DISCOUNT FACTORS
# =============================================================================
print(f"\n{datetime.now()}: STEP 9 - Calculating Discount Factors")
print("-"*80)

print(f"Eta: {eta}, Rho: {rho}")

discount_config = DiscountingConfig(
    discount_type=discount_type,
    eta=eta,
    rho=rho,
    discrete=conf["global_parameters"]["discrete_discounting"]
)

discount_factors = calculate_stream_discount_factors_per_scenario(
    global_consumption_no_pulse=global_consumption,
    population=pop,
    eta=discount_config.eta,
    rho=discount_config.rho,
    pulse_year=pulse_year,
    discount_type=discount_type,
    fair_aggregation=fair_agg_for_marginal_damages,
    fair_dims=None,  # Auto-detect
    discrete=discount_config.discrete,
    ext_end_year=ext_end_year
)

print(f"\n✓ Discount factors calculated")
print(f"  Dimensions: {discount_factors.dims}")
print(f"  Shape: {discount_factors.sizes}")


2025-10-08 02:12:32.633453: STEP 9 - Calculating Discount Factors
--------------------------------------------------------------------------------
Eta: 2.0, Rho: 0.0001

✓ Discount factors calculated
  Dimensions: ('fair_aggregation', 'discount_type', 'ssp', 'model', 'year')
  Shape: Frozen({'fair_aggregation': 3, 'discount_type': 1, 'ssp': 5, 'model': 2, 'year': 81})


In [22]:
# =============================================================================
# STEP 10: CALCULATE SCC
# =============================================================================
print(f"\n{datetime.now()}: STEP 10 - Calculating SCC")
print("-"*80)

scc = calculate_scc_with_uncertainty(
    marginal_damages=marginal_damages,
    discount_factors=discount_factors,
    discount_type=discount_type,
    fair_aggregation_methods=fair_agg_for_marginal_damages,
    include_median='median' in fair_agg,
    pulse_year=pulse_year
)

print(f"\n✓ SCC calculated")
print(f"  Dimensions: {scc.dims}")
print(f"  Shape: {scc.sizes}")

# Convert to DataFrame for easier visualization
print("\nConverting SCC to DataFrame for easier visualization...")
scc_df = scc.to_dataframe(name='scc').reset_index()

# Display basic info
print(f"  DataFrame shape: {scc_df.shape}")
print(f"  Columns: {list(scc_df.columns)}")

# Show first few rows
print("\n  First 10 rows:")
print(scc_df.head(10))


2025-10-08 02:13:00.411797: STEP 10 - Calculating SCC
--------------------------------------------------------------------------------

✓ SCC calculated
  Dimensions: ('fair_aggregation', 'discount_type', 'ssp', 'model', 'weitzman_parameter', 'rcp', 'gcm', 'gas', 'pulse_year', 'pulse_years')
  Shape: Frozen({'fair_aggregation': 3, 'discount_type': 1, 'ssp': 5, 'model': 2, 'weitzman_parameter': 5, 'rcp': 2, 'gcm': 2, 'gas': 1, 'pulse_year': 1, 'pulse_years': 1})

Converting SCC to DataFrame for easier visualization...
  DataFrame shape: (600, 11)
  Columns: ['fair_aggregation', 'discount_type', 'ssp', 'model', 'weitzman_parameter', 'rcp', 'gcm', 'gas', 'pulse_year', 'pulse_years', 'scc']

  First 10 rows:
  fair_aggregation discount_type   ssp   model weitzman_parameter     rcp  \
0               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
1               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
2               ce  naive_ramsey  ssp1  dummy1        

In [19]:
import yaml
import xarray as xr

conf = yaml.safe_load(open('../../dscim-testing/configs/dummy_config.yaml'))
fair_agg = conf['global_parameters']['fair_aggregation']
print('Config fair_aggregation:', fair_agg)

# Check what we're filtering
fair_agg_impl = [agg for agg in fair_agg if agg != 'median_params']
print('Implemented (no median_params):', fair_agg_impl)

Config fair_aggregation: ['ce', 'median', 'mean', 'gwr_mean', 'median_params']
Implemented (no median_params): ['ce', 'median', 'mean', 'gwr_mean']


In [28]:
# Convert to DataFrame for easier visualization
print("\nConverting SCC to DataFrame for easier visualization...")
scc_df = scc.to_dataframe(name='scc').reset_index()

# Display basic info
print(f"  DataFrame shape: {scc_df.shape}")
print(f"  Columns: {list(scc_df.columns)}")

# Show first few rows
print("\n  First 10 rows:")
print(scc_df.head(10))




Converting SCC to DataFrame for easier visualization...
  DataFrame shape: (4000, 12)
  Columns: ['fair_aggregation', 'discount_type', 'ssp', 'model', 'weitzman_parameter', 'rcp', 'gcm', 'simulation', 'gas', 'pulse_year', 'pulse_years', 'scc']

  First 10 rows:
  fair_aggregation discount_type   ssp   model weitzman_parameter     rcp  \
0               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
1               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
2               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
3               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
4               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
5               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
6               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
7               ce  naive_ramsey  ssp1  dummy1                0.1  dummy1   
8               ce  naive_ramsey  ssp1  dumm

In [ ]:

  Reference DataFrame shape: (300, 8)
  Reference columns: ['fair_aggregation', 'ssp', 'model', 'weitzman_parameter', 'discount_type', 'rcp', 'gas', 'scc_ref']

  First 10 rows of reference:
  fair_aggregation   ssp   model weitzman_parameter discount_type     rcp  \
0               ce  ssp2  dummy1                0.1  naive_ramsey  dummy1   
1               ce  ssp2  dummy1                0.1  naive_ramsey  dummy2   
2               ce  ssp2  dummy1                1.0  naive_ramsey  dummy1   
3               ce  ssp2  dummy1                1.0  naive_ramsey  dummy2   
4               ce  ssp2  dummy1               0.25  naive_ramsey  dummy1   
5               ce  ssp2  dummy1               0.25  naive_ramsey  dummy2   
6               ce  ssp2  dummy1               0.01  naive_ramsey  dummy1   
7               ce  ssp2  dummy1               0.01  naive_ramsey  dummy2   
8               ce  ssp2  dummy1              0.001  naive_ramsey  dummy1   
9               ce  ssp2  dummy1              0.001  naive_ramsey  dummy2   

         gas      scc_ref  
0  dummy_gas -1015.752377  
1  dummy_gas   960.958811  
2  dummy_gas  -975.130019  
3  dummy_gas   934.127323  
4  dummy_gas -1015.752377  
5  dummy_gas   960.958811  
6  dummy_gas -1015.752377  
7  dummy_gas   960.958811  
8  dummy_gas -1015.752377  

In [24]:
reference_scc = xr.open_dataset(ref_path)
reference_scc.to_dataframe(name='scc').reset_index()

TypeError: Dataset.to_dataframe() got an unexpected keyword argument 'name'

In [26]:
if "scc" in reference_scc:
    ref_values = reference_scc["scc"]
    ref_df = ref_values.to_dataframe(name='scc_ref').reset_index()

    print(f"\n  Reference DataFrame shape: {ref_df.shape}")
    print(f"  Reference columns: {list(ref_df.columns)}")
    print(f"\n  First 10 rows of reference:")
    print(ref_df.head(10))


  Reference DataFrame shape: (300, 8)
  Reference columns: ['fair_aggregation', 'ssp', 'model', 'weitzman_parameter', 'discount_type', 'rcp', 'gas', 'scc_ref']

  First 10 rows of reference:
  fair_aggregation   ssp   model weitzman_parameter discount_type     rcp  \
0               ce  ssp2  dummy1                0.1  naive_ramsey  dummy1   
1               ce  ssp2  dummy1                0.1  naive_ramsey  dummy2   
2               ce  ssp2  dummy1                1.0  naive_ramsey  dummy1   
3               ce  ssp2  dummy1                1.0  naive_ramsey  dummy2   
4               ce  ssp2  dummy1               0.25  naive_ramsey  dummy1   
5               ce  ssp2  dummy1               0.25  naive_ramsey  dummy2   
6               ce  ssp2  dummy1               0.01  naive_ramsey  dummy1   
7               ce  ssp2  dummy1               0.01  naive_ramsey  dummy2   
8               ce  ssp2  dummy1              0.001  naive_ramsey  dummy1   
9               ce  ssp2  dummy1      

In [19]:

  Reference DataFrame shape: (300, 8)
  Reference columns: ['fair_aggregation', 'ssp', 'model', 'weitzman_parameter', 'discount_type', 'rcp', 'gas', 'scc_ref']

  First 10 rows of reference:
  fair_aggregation   ssp   model weitzman_parameter discount_type     rcp  \
0               ce  ssp2  dummy1                0.1  naive_ramsey  dummy1   
1               ce  ssp2  dummy1                0.1  naive_ramsey  dummy2   
2               ce  ssp2  dummy1                1.0  naive_ramsey  dummy1   
3               ce  ssp2  dummy1                1.0  naive_ramsey  dummy2   
4               ce  ssp2  dummy1               0.25  naive_ramsey  dummy1   
5               ce  ssp2  dummy1               0.25  naive_ramsey  dummy2   
6               ce  ssp2  dummy1               0.01  naive_ramsey  dummy1   
7               ce  ssp2  dummy1               0.01  naive_ramsey  dummy2   
8               ce  ssp2  dummy1              0.001  naive_ramsey  dummy1   
9               ce  ssp2  dummy1              0.001  naive_ramsey  dummy2   

         gas      scc_ref  
0  dummy_gas -1015.752377  
1  dummy_gas   960.958811  
2  dummy_gas  -975.130019  
3  dummy_gas   934.127323  
4  dummy_gas -1015.752377  
5  dummy_gas   960.958811  
6  dummy_gas -1015.752377  
7  dummy_gas   960.958811  
8  dummy_gas -1015.752377  
9  dummy_gas   960.958811  

<xarray.Dataset> Size: 3kB
Dimensions:             (fair_aggregation: 5, ssp: 3, model: 2,
                         weitzman_parameter: 5, discount_type: 1, rcp: 2, gas: 1)
Coordinates:
  * model               (model) <U6 48B 'dummy1' 'dummy2'
  * ssp                 (ssp) <U4 48B 'ssp2' 'ssp3' 'ssp4'
  * discount_type       (discount_type) <U12 48B 'naive_ramsey'
  * rcp                 (rcp) <U6 48B 'dummy1' 'dummy2'
  * gas                 (gas) <U9 36B 'dummy_gas'
  * weitzman_parameter  (weitzman_parameter) <U5 100B '0.1' '1.0' ... '0.001'
  * fair_aggregation    (fair_aggregation) <U13 260B 'ce' ... 'median_params'
Data variables:
    scc                 (fair_aggregation, ssp, model, weitzman_parameter, discount_type, rcp, gas) float64 2kB ...
Attributes: (12/49)
    sector_path:                    None
    save_path:                      ./dummy_data/results/AR6_ssp/dummy_not_co...
    gdppc_bottom_code:              39.39265060424805
    subset_dict:                    {'ssp': ['ssp2', 'ssp3', 'ssp4']}
    econ_vars:                      <dscim.menu.simple_storage.EconVars objec...
    climate:                        <dscim.menu.simple_storage.Climate object...
    ...                             ...
    ecs_mask_path:                  ./dummy_data/climate/parameter_filters_tr...
    ecs_mask_name:                  None
    anomaly_vars:                   ['temperature', 'gmsl']
    path:                           ./dummy_data/econ/integration-econ-bc39.zarr
    machine:                        MacBookPro.home.local
    commit:                         b'9b7125b'

In [2]:
import sys
import yaml
import xarray as xr
#sys.path.insert(0, '.')

# Load test data
fair_combined = xr.open_dataset('../../dscim-testing/dummy_data/fair/pulse/fair_combined_dummy.nc4')
print('FAIR data dimensions:', dict(fair_combined.dims))
print('FAIR data coords:', list(fair_combined.coords))

# Check if pulse_years exists
if 'pulse_years' in fair_combined.dims:
    print('pulse_years dimension found in FAIR data')
    print('pulse_years values:', fair_combined.pulse_years.values)
if 'pulse_year' in fair_combined.dims:
    print('pulse_year dimension found in FAIR data')
    print('pulse_year values:', fair_combined.pulse_year.values)
if 'pulse_years' in fair_combined.coords:
    print('pulse_years coordinate found')
if 'pulse_year' in fair_combined.coords:
    print('pulse_year coordinate found')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/dummy_data/fair/pulse/fair_combined_dummy.nc4'

In [5]:
ls /Users/sebastiancadavidsanchez/Documents/Github/cil/dscim-new/dscim-testing/dummy_data/climate

GMTanom_all_temp_2001_2010_smooth.csv  conversion.nc4
ar6_fair162_sim.nc                     scenario_gmsl.nc4
coastal_gmsl_v0.20.zarr/
